# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

In [2]:
import datetime
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('sparkSQL') \
                    .getOrCreate()

In [4]:
path = r"data/sparkify_log_small.json"
df = spark.read.json(path)

In [5]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
df.createOrReplaceTempView('dfSQL')

In [7]:
spark.sql("""
          SELECT *
          FROM dfSQL
          LIMIT 5
          """
          ).show()

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

# Question 1

Which page did user id ""(empty string) NOT visit?

In [18]:
allPages = spark.sql('''
                     SELECT DISTINCT page AS pages
                     FROM dfSQL
                     '''
                     )

visitedPages = spark.sql("""
                       SELECT DISTINCT page AS pages
                       FROM dfSQL
                       WHERE userID == ''
                       """
                       )

In [19]:
allPages.show()
visitedPages.show()

+----------------+
|           pages|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+

+-----+
|pages|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [24]:
notVisitedPages = spark.sql('''
                            SELECT *
                            FROM(
                                SELECT
                                    DISTINCT page AS pages
                                    FROM dfSQL
                            ) AS allPages
                            LEFT JOIN(
                                SELECT
                                    DISTINCT page AS pages
                                    FROM dfSQL
                                    WHERE dfSQL.userID == ''
                            ) AS visitedPages
                            ON allPages.pages = visitedPages.pages
                            WHERE visitedPages.pages IS NULL
                            '''
                            )
notVisitedPages.show()

+----------------+-----+
|           pages|pages|
+----------------+-----+
|Submit Downgrade| null|
|       Downgrade| null|
|          Logout| null|
|   Save Settings| null|
|        Settings| null|
|        NextSong| null|
|         Upgrade| null|
|           Error| null|
|  Submit Upgrade| null|
+----------------+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

In [ ]:
# i think it is more simple and universal language

# Question 3

How many female users do we have in the data set?

In [28]:
# TODO: write your code to answer question 3
spark.sql(
    '''
    SELECT
        COUNT(DISTINCT userID)
    FROM dfSQL
    WHERE gender = 'F'
    '''
).show()

+----------------------+
|count(DISTINCT userID)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [32]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [35]:
spark.sql(
    '''
    SELECT DISTINCT page
    FROM dfSQL
    '''
).show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [37]:
# TODO: write your code to answer question 4
spark.sql(
    '''
    SELECT
        artist,
        COUNT(1) AS plays
    FROM dfSQL
    WHERE page = 'NextSong'
    GROUP BY artist
    ORDER BY plays DESC
    LIMIT 1
    '''
).show()

+--------+-----+
|  artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [51]:
# TODO: write your code to answer question 5
spark.sql(
    '''
    SELECT
        userID,
        page,
        COUNT(1) AS frequency
    FROM dfSQL
    WHERE
        page  = 'Home'
        OR page = 'NextSong'
        AND userID <> ''
    GROUP BY
        userID,
        page
    ORDER BY
        userID,
        page
    '''
).show()

+------+--------+---------+
|userID|    page|frequency|
+------+--------+---------+
|      |    Home|      187|
|    10|NextSong|        2|
|   100|    Home|        3|
|   100|NextSong|        5|
|  1000|NextSong|        1|
|  1003|    Home|        2|
|  1003|NextSong|        1|
|  1005|NextSong|        1|
|  1006|NextSong|        3|
|  1017|    Home|        2|
|  1017|NextSong|       10|
|  1019|    Home|        5|
|  1019|NextSong|       22|
|  1020|    Home|        1|
|  1020|NextSong|        4|
|  1022|NextSong|        2|
|  1025|    Home|        1|
|  1025|NextSong|        4|
|  1030|NextSong|        1|
|  1035|    Home|        6|
+------+--------+---------+
only showing top 20 rows



In [55]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql(
    '''
    SELECT
        userID,
        page,
        ts,
        CASE
            WHEN page = 'Home' THEN 1
            ELSE 0 END
            AS is_home
        FROM dfSQL
        WHERE page = 'Home' or page = 'NextSong'
    '''
)

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql(
    '''
    SELECT
        *,
        SUM(is_home)
        OVER(PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
        AS period
    FROM is_home_table
    '''
)

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
spark.sql(
    '''
    SELECT
        AVG(count_results)
    FROM(
        SELECT
            COUNT(1) AS count_results
        FROM period_table
        GROUP BY
            userID,
            period,
            page
        HAVING page = 'NextSong'
    ) AS counts
    '''
).show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

